# Tensorflow Implementation of VGG Face Detector

Objective: Convert the VGG face descriptor model http://www.robots.ox.ac.uk/~vgg/software/vgg_face/ to Tensorflow format.
<br>
Output: Final Tensorflow classifier model trained on the gender dataset

In [1]:
import scipy.io as sio
import tensorflow as tf
from keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Reshape, ZeroPadding2D, Convolution2D, MaxPooling2D, Dense, Dropout, Softmax, Flatten, Activation, BatchNormalization

Using TensorFlow backend.


## Read .mat file for Model Description

In [2]:
def load_mat():
    mat = sio.loadmat('data/vgg_face.mat', struct_as_record=False)
    return mat

mat = load_mat()
net = mat['net'][0][0]
mat_model = net.layers
mat_model_layers = mat_model[0]
num_mat_layers = mat_model_layers.shape[0]
for layer in mat_model_layers:
    print(layer[0][0].name)

['conv1_1']
['relu1_1']
['conv1_2']
['relu1_2']
['pool1']
['conv2_1']
['relu2_1']
['conv2_2']
['relu2_2']
['pool2']
['conv3_1']
['relu3_1']
['conv3_2']
['relu3_2']
['conv3_3']
['relu3_3']
['pool3']
['conv4_1']
['relu4_1']
['conv4_2']
['relu4_2']
['conv4_3']
['relu4_3']
['pool4']
['conv5_1']
['relu5_1']
['conv5_2']
['relu5_2']
['conv5_3']
['relu5_3']
['pool5']
['fc6']
['relu6']
['dropout6']
['fc7']
['relu7']
['dropout7']
['fc8']
['softmax']


## VGG Architecture Implementation
We can read the model described in the .mat file and build our tensorflow/keras model using clues. This way we can avoid reproducing it by hand. Additionally, this saves us the trouble of later applying the model weights from the .mat file to our tensorflow/keras model.

In [3]:
def vgg_tf():
    model = Sequential()

    for i in range(num_mat_layers):
        mat_model_layer = mat_model_layers[i][0][0].name[0]
        if mat_model_layer.find("conv") == 0 or mat_model_layer.find("fc") == 0:
            weights = mat_model_layers[i][0,0].weights
            weights_shape = weights[0][0].shape
            filter_x = weights_shape[0]; filter_y = weights_shape[1]
            number_of_filters = weights_shape[3]

            if mat_model_layer.find("conv") == 0:
                print("ZeroPadding2D((1,1))")
                if i == 0:
                    model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
                else:
                    model.add(ZeroPadding2D((1,1)))

            print("Convolution2D(",number_of_filters,", (",filter_x,", ",filter_y,"), name='",mat_model_layer,"')")
            model.add(Convolution2D(number_of_filters, (filter_x, filter_y), name= mat_model_layer))

        else:
            if mat_model_layer.find("relu") == 0:
                print("Activation('relu', name=",mat_model_layer)
                model.add(Activation('relu', name=mat_model_layer))
            elif mat_model_layer.find("dropout") == 0:
                print("Dropout(0.5, name=",mat_model_layer,")")
                model.add(Dropout(0.5, name=mat_model_layer))
            elif mat_model_layer.find("pool") == 0:
                print("MaxPooling2D((2,2), strides=(2,2), name=",mat_model_layer,")")
                model.add(MaxPooling2D((2,2), strides=(2,2), name=mat_model_layer))
            elif mat_model_layer.find("softmax") == 0:
                print("Activation('softmax', name=",mat_model_layer,")")
                model.add(Activation('softmax', name=mat_model_layer))
            else:
                print("-->",mat_model_layer)
    return model

Save model to .h5 file for use later.

In [4]:
model = vgg_tf()
model.save('data/vgg_face.h5')

ZeroPadding2D((1,1))
Convolution2D( 64 , ( 3 ,  3 ), name=' conv1_1 ')
Activation('relu', name= relu1_1
ZeroPadding2D((1,1))
Convolution2D( 64 , ( 3 ,  3 ), name=' conv1_2 ')
Activation('relu', name= relu1_2
MaxPooling2D((2,2), strides=(2,2), name= pool1 )
ZeroPadding2D((1,1))
Convolution2D( 128 , ( 3 ,  3 ), name=' conv2_1 ')
Activation('relu', name= relu2_1
ZeroPadding2D((1,1))
Convolution2D( 128 , ( 3 ,  3 ), name=' conv2_2 ')
Activation('relu', name= relu2_2
MaxPooling2D((2,2), strides=(2,2), name= pool2 )
ZeroPadding2D((1,1))
Convolution2D( 256 , ( 3 ,  3 ), name=' conv3_1 ')
Activation('relu', name= relu3_1
ZeroPadding2D((1,1))
Convolution2D( 256 , ( 3 ,  3 ), name=' conv3_2 ')
Activation('relu', name= relu3_2
ZeroPadding2D((1,1))
Convolution2D( 256 , ( 3 ,  3 ), name=' conv3_3 ')
Activation('relu', name= relu3_3
MaxPooling2D((2,2), strides=(2,2), name= pool3 )
ZeroPadding2D((1,1))
Convolution2D( 512 , ( 3 ,  3 ), name=' conv4_1 ')
Activation('relu', name= relu4_1
ZeroPadding2D((

## Construct Classifier 

In [5]:
def classier_model(input_dim, num_classes):
    classifier = Sequential()
    classifier.add(Dense(100, input_dim=input_dim)) #input dim
    classifier.add(BatchNormalization())
    classifier.add(Activation('tanh'))
    classifier.add(Dropout(0.3))
    classifier.add(Dense(20))
    classifier.add(BatchNormalization())
    classifier.add(Activation('tanh'))
    classifier.add(Dropout(0.1))
    classifier.add(Dense(num_classes))
    classifier.add(Activation('softmax'))
    classifier.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
    return classifier

In [6]:
classifier = classier_model(128, 2)

## Load Images

In [10]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from PIL import Image
import numpy as np
from tqdm import tqdm
import os

In [25]:
def pad(img):
    ht, wd, cc= img.shape
    ww = 224
    hh = 224
    color = (0,0,0)
    result = np.full((hh,ww,cc), color, dtype=np.uint8)
    xx = (ww - wd) // 2
    yy = (hh - ht) // 2
    result[yy:yy+ht, xx:xx+wd] = img
    return result

def load_img_file(path, model):
    img = Image.open(path)
#     img = load_img(path)
    x = img_to_array(img) 
    x = pad(x)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    x = model(x)
    return x

def load_data(path, model):
    x = []
    y = []
    labels = {}
    folders = os.listdir(path)
    for i in tqdm(range(len(folders))):
        labels[i] = folders[i]
        label_path = path + '/' + folders[i]
        images = os.listdir(label_path)
        for image in images:
            img = load_img_file(label_path + '/' + image, model)
            x.append(np.squeeze(K.eval(img)).tolist())
            y.append(i)
    return x, y, labels

In [26]:
X_train, y_train, labels = load_data('data/combined/aligned', model)
np.save('data/X_train.npy', X_train)
np.save('data/y_train.npy', y_train)
np.save('data/labels.npy', labels)

100%|████████████████████████████████████████████████████████████████████████████████| 140/140 [20:51<00:00,  8.94s/it]

{0: '01_F', 1: '01_M', 2: '02_F', 3: '02_M', 4: '03_F', 5: '03_M', 6: '04_F', 7: '04_M', 8: '05_F', 9: '05_M', 10: '06_F', 11: '06_M', 12: '07_F', 13: '07_M', 14: '08_F', 15: '08_M', 16: '09_F', 17: '09_M', 18: '10_F', 19: '10_M', 20: '11_F', 21: '11_M', 22: '12_F', 23: '12_M', 24: '13_F', 25: '13_M', 26: '14_F', 27: '14_M', 28: '15_F', 29: '15_M', 30: '16_F', 31: '16_M', 32: '17_F', 33: '17_M', 34: '18_F', 35: '18_M', 36: '19_F', 37: '19_M', 38: '20_F', 39: '20_M', 40: '21_F', 41: '21_M', 42: '22_F', 43: '22_M', 44: '23_F', 45: '23_M', 46: '24_F', 47: '24_M', 48: '25_F', 49: '25_M', 50: '26_F', 51: '26_M', 52: '27_F', 53: '27_M', 54: '28_F', 55: '28_M', 56: '29_F', 57: '29_M', 58: '30_F', 59: '30_M', 60: '31_F', 61: '31_M', 62: '32_F', 63: '32_M', 64: '33_F', 65: '33_M', 66: '34_F', 67: '34_M', 68: '35_F', 69: '35_M', 70: '36_F', 71: '36_M', 72: '37_F', 73: '37_M', 74: '38_F', 75: '38_M', 76: '39_F', 77: '39_M', 78: '40_F', 79: '40_M', 80: '41_F', 81: '41_M', 82: '42_F', 83: '42_M', 8

In [27]:
X_test, y_test, labels = load_data('data/combined/valid', model)
np.save('data/X_test.npy', X_test)
np.save('data/y_test.npy', y_test)

 80%|████████████████████████████████████████████████████████████████                | 112/140 [02:34<00:38,  1.38s/it]


KeyboardInterrupt: 

## Train Classifier

In [ ]:
def train()

## Test Model

## Results